In [ ]:
#%matplotlib notebook
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
import numpy as np
import subprocess
import aplpy
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from astropy.table import Table
from astropy.io.fits import getheader
from astropy.convolution import convolve
from astropy.convolution import Gaussian2DKernel
from tqdm import tqdm_notebook

# parallel processor
from utilities import parallel_process                
                
import warnings
from astropy.utils.exceptions import AstropyWarning
warnings.simplefilter('ignore', category=AstropyWarning)
from astropy import log
log.setLevel('WARN')

In [ ]:
def cp_results(outpath):
    print('Copying files...')
    
    os.system(f'find {outpath}/PSZ* -name "*vtp.png" -exec cp -t {outpath}/pngs/images/ ' '{} \;')

In [ ]:
def show_sources(name, outpath, zoom=False, window=7):
    # check for files
    # check for detections
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        srcs = f'{outpath}/{name}/{name}_vtp.detect'
    else:
        pass # make the images even if we don't have any detections
    # events image
    if os.path.isfile(f'{outpath}/{name}/{name}_img_50-600.fits'):
        evnts = f'{outpath}/{name}/{name}_img_50-600.fits'
    else:
        return
    
    # check for imaging -- sdss first
    if os.path.isfile(f'{outpath}/{name}/{name}_DESstack_r.fits'):
        survey = 'DES'
        optimage = True
        img = f'{outpath}/{name}/{name}_DESstack_r.fits'
    elif os.path.isfile(f'{outpath}/{name}/{name}_DECaLSstack_r.fits'):
        survey = 'DECaLS'
        optimage = True
        img = f'{outpath}/{name}/{name}_DECaLSstack_r.fits'
    elif os.path.isfile(f'{outpath}/{name}/{name}_SDSSstack_i.fits'):
        survey = 'SDSS'
        optimage = True
        img = f'{outpath}/{name}/{name}_SDSSstack_i.fits'
    elif os.path.isfile(f'{outpath}/{name}/{name}_PS1stack_i.fits'):
        survey = 'PS1'
        optimage = True
        img = f'{outpath}/{name}/{name}_PS1stack_i.fits'
    else:
        optimage = False   
    
    info = Table.read(f'{outpath}/{name}/{name}.info', format='ascii.fast_csv')
    
    # show the figure
    gc = aplpy.FITSFigure(evnts, figsize=(10, 10))
    gc.show_grayscale(vmin=0, pmax=100, stretch='linear', interpolation='none')
    
    # recenter on the PSZ position, and make a n'x n' window
    if zoom:
        gc.recenter(info['RA'], info['DEC'], window / 60)
    
    
    # now we need to read the individual detections
#     detects = Table.read(srcs, hdu=1)
    # add all the sources -- the float is the pixscale in deg
#     r1 = detects['R'].data[:,0] * 6.548089E-04
#     r2 = detects['R'].data[:,1] * 6.548089E-04
#     gc.show_ellipses(detects['RA'], detects['DEC'], r1, r2, detects['ROTANG'], 
#                      coords_frame='world', edgecolor='cyan')
    try:
        gc.show_regions(f'{outpath}/{name}/{name}_vtp.reg')
    except FileNotFoundError:
        pass # make the images even if there aren't any detections
        
    # add PSZ info and circles
    gc.show_circles(info['RA'], info['DEC'], 2 / 60,
                    linestyle='--', edgecolor='#e24a33', facecolor='none',
                    path_effects=[pe.Stroke(linewidth=1.2, foreground='white'),
                                  pe.Normal()])
    gc.show_circles(info['RA'], info['DEC'], 5 / 60,
                    linestyle='-', edgecolor='#e24a33', facecolor='none',
                    path_effects=[pe.Stroke(linewidth=1.2, foreground='white'),
                                  pe.Normal()])
    gc.show_markers(info['RA'], info['DEC'],
                    marker='*', s=150, layer='psz', edgecolor='#e24a33',
                    path_effects=[pe.Stroke(linewidth=1.2,
                                            foreground='white'), pe.Normal()])

    # write the exposure time
    exp_time = getheader(evnts)['EXPOSURE']
    text = f'exp time: {exp_time:.2f}s'
    xo, yo = (0.05, 0.05)
    gc.add_label(xo, yo, text, relative=True, fontsize=18, color='white', horizontalalignment='left')

    # write redshift
    ztext = f'z: {info["REDSHIFT"][0]:.3f}'
    gc.add_label(xo, yo + 0.03, ztext, relative=True, fontsize=18, color='white', horizontalalignment='left')
    
    # write legend
    xo, yo = (0.95, 0.05)
    gc.add_label(xo, yo, 'Extended', relative=True, fontsize=18, color='magenta', 
                 horizontalalignment='right')
    gc.add_label(xo, yo + 0.03, 'P-Source', relative=True, fontsize=18, color='yellow', 
                 horizontalalignment='right')
    if zoom:
        gc.save(f'{outpath}/{name}/{name}_XRT_vtp_zoom.png', dpi=90)
    else:
        gc.save(f'{outpath}/{name}/{name}_XRT_vtp.png', dpi=90)

    gc.close()
    
    ### optical imaging ###    
    if optimage:
        # make sure the links aren't broken
        if os.path.exists(f'{outpath}/{name}/{name}_{survey}stack.jpg'):
            ending = 'stack.jpg'
        elif os.path.exists(f'{outpath}/{name}/{name}_{survey}stack_irg.tiff'):
            ending = 'stack_irg.tiff'
        else:
            return
            
        # show the figure
        gc = aplpy.FITSFigure(img, figsize=(10, 10))
        try:
            gc.show_rgb(f'{outpath}/{name}/{name}_{survey}{ending}')
        except FileNotFoundError:
            gc.show_grayscale(stretch='arcsinh', pmin=1, pmax=98)
            gc.set_theme('publication')

        # recenter on the PSZ position, and make a n'x n' window
        if zoom:
            gc.recenter(info['RA'], info['DEC'], window / 60)
            
        #gc.set_tick_labels_format(xformat='hh:mm:ss', yformat='dd:mm')
        #gc.set_tick_labels_size('small')
       
        try:
            gc.show_regions(f'{outpath}/{name}/{name}_vtp.reg')
        except FileNotFoundError:
            pass # make the images even if there aren't any detections
        
        # add PSZ info and circles
        gc.show_circles(info['RA'], info['DEC'], 2 / 60,
                        linestyle='--', edgecolor='#e24a33', facecolor='none',
                        path_effects=[pe.Stroke(linewidth=1.2, foreground='white'),
                                      pe.Normal()])
        gc.show_circles(info['RA'], info['DEC'], 5 / 60,
                        linestyle='-', edgecolor='#e24a33', facecolor='none',
                        path_effects=[pe.Stroke(linewidth=1.2, foreground='white'),
                                      pe.Normal()])
        gc.show_markers(info['RA'], info['DEC'],
                        marker='*', s=150, layer='psz', edgecolor='#e24a33',
                        path_effects=[pe.Stroke(linewidth=1.2,
                                                foreground='white'), pe.Normal()])

        xo, yo = (0.05, 0.05)
        # write the exposure time
        gc.add_label(xo, yo, text, relative=True, fontsize=18, color='white', horizontalalignment='left')

        # write redshift
        gc.add_label(xo, yo + 0.03, ztext, relative=True, fontsize=18, color='white', horizontalalignment='left')
        
        # write legend
        xo, yo = (0.95, 0.05)
        gc.add_label(xo, yo, 'Extended', relative=True, fontsize=18, color='magenta', 
                 horizontalalignment='right')
        gc.add_label(xo, yo + 0.03, 'P-Source', relative=True, fontsize=18, color='yellow', 
                 horizontalalignment='right')
        if zoom:
            gc.save(f'{outpath}/{name}/{name}_OP_vtp_zoom.png', dpi=90)
        else:
            gc.save(f'{outpath}/{name}/{name}_OP_vtp.png', dpi=90)

        gc.close()
        
    return


In [ ]:
# get file data
data = load_PSZcatalog()
data = data.sort_values('NAME')

outpath = './data'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
parallel_process(arr, show_sources, use_kwargs=True, n_jobs=6)

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath, 'zoom':True} for n in data['NAME']]
parallel_process(arr, show_sources, use_kwargs=True, n_jobs=6)

cp_results(outpath)

In [ ]:
# outpath = './data_full'
# name = 'PSZ2_G287.96-32.99'
# show_sources(name, outpath)